In [1]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/RAG-project-SMILES-2024-"
sys.path.insert(0, BASE_DIR)

from src.utils import evaluate_retriever
from src.utils import RetrieverMetrics, save_retriever_trial_log, prepare_retriever_configs, load_benchmarks_df

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# !!! TO CHANGE !!!
TRIAL = 5
BENCHMARKS_MAXSIZE = 10000
BENCHMARKS_INFO = {'mtssquad': {'db': 'v2', 'table': 'v1'}}

RETRIEVER_PARAMS = {
    "model_path": f"{BASE_DIR}/models/intfloat/multilingual-e5-small",
    "densedb_kwargs": {'metadata': {"hnsw:space": "ip"}},
    "model_kwargs": {'device':'cuda'},
    "encode_kwargs": {'normalize_embeddings': True, 'prompt': 'query: '},
    "params": {'fetch_k': 50, 'threshold': 0.18, 'max_k': 3}
}
# !!! TO CHANGE !!!

SAVE_LOGDIR = f'./logs/trial{TRIAL}'
SAVE_HYPERPARAMS = f'{SAVE_LOGDIR}/hyperparams.json'
SAVE_RETRIEVERCACHE = f'{SAVE_LOGDIR}/retriever_cache.json'

In [22]:
retrivers_config, benchmarks_path = prepare_retriever_configs(BASE_DIR, BENCHMARKS_INFO, RETRIEVER_PARAMS)
metrics = RetrieverMetrics()

In [23]:
benchmarks_df = load_benchmarks_df(benchmarks_path, BENCHMARKS_MAXSIZE)

In [24]:
retriever_scores, retriever_cache, _ = evaluate_retriever(benchmarks_df, retrivers_config, metrics)

No sentence-transformers model found with name /trinity/home/team06/workspace/mikhail_workspace/RAG-project-SMILES-2024-/models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


mtssquad


  0%|          | 0/10000 [00:00<?, ?it/s]/trinity/home/team06/workspace/mikhail_workspace/RAG-project-SMILES-2024-/src/utils/evaluation_metrics.py:37: RuntimeWarning: invalid value encountered in scalar divide
  return (2 * self.precision(pred_cands, gold_cands, k) * self.recall(pred_cands, gold_cands, k)) / (self.precision(pred_cands, gold_cands, k) + self.recall(pred_cands, gold_cands, k))
100%|██████████| 10000/10000 [02:39<00:00, 62.57it/s, MRR=0.802, mAP=0.802, Recall=0.862, Precision=0.287, F1=0.431]


In [25]:
save_retriever_trial_log(SAVE_LOGDIR, retriever_scores, SAVE_HYPERPARAMS, SAVE_RETRIEVERCACHE,
                         retriever_cache, BENCHMARKS_INFO, BENCHMARKS_MAXSIZE, RETRIEVER_PARAMS)